## **Object Detection and Annotation with YOLOv8** ##

This notebook demonstrates how to use the YOLOv8 model to perform object detection on a video, filter detections to identify greyhounds, and save the annotated video. The code leverages the cv2 library for video processing, ultralytics for YOLOv8, and supervision for object annotation.

In [ ]:
import cv2
import os
import numpy as np
from ultralytics import YOLO
import supervision as sv


cv2: OpenCV library for video processing.

os: For interacting with the operating system (e.g., checking directories).

numpy: For numerical operations.

YOLO from ultralytics: YOLOv8 model for object detection.

supervision as sv: For object detection and annotation.

**File Paths and Model Initialization**

In [ ]:
GREYHOUND_VIDEO = "../data/raw_videos/20240722NOTG08_V.mp4"
model = YOLO("yolov8n.pt")

GREYHOUND_VIDEO: Path to the input video file.

model: Initializes the YOLOv8 model with the pre-trained weights yolov8n.pt

**Video and Output Setup**

In [ ]:
cap = cv2.VideoCapture(GREYHOUND_VIDEO)

output_dir = "./output"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
if not os.access(output_dir, os.W_OK):
    raise PermissionError(f"Write permission denied for the directory {output_dir}")

output_path = os.path.join(output_dir, "object_counting_output_v8_3.mp4")
assert cap.isOpened(), "Error reading video file"

w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

fourcc_code = cv2.VideoWriter_fourcc(*"mp4v")
video_writer = cv2.VideoWriter(output_path, fourcc_code, fps, (w, h))

if not video_writer.isOpened():
    raise IOError(f"Error initializing video writer with path {output_path}")


cap: Opens the input video file.

output_dir: Directory to save the output video. Creates the directory if it does not exist.

output_path: Path for the output video file.

Checks if the video file is opened and initializes VideoWriter with appropriate codec and properties.

**Detection Filtering Function**

In [ ]:
def filter_greyhound_detections(detections):
    greyhound_class_id = 16
    mask = detections.class_id == greyhound_class_id
    filtered_detections = sv.Detections(
        xyxy=detections.xyxy[mask],
        confidence=detections.confidence[mask],
        class_id=detections.class_id[mask]
    )
    return filtered_detections


filter_greyhound_detections: Custom function to filter detections to only include greyhounds (class ID 16).

**Video Processing Loop**

Initializes BoxAnnotator for annotating bounding boxes.
Reads frames from the video file.
Performs object detection using YOLOv8.
Extracts and filters detections to include only greyhounds.
Annotates frames with bounding boxes and writes them to the output video file.
Releases video resources and saves the processed video.

In [ ]:
box_annotator = sv.BoxAnnotator(thickness=4)
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Video frame is empty or video processing has been successfully completed.")
        break

    results = model(frame, imgsz=1280)
    boxes = results[0].boxes.xyxy.cpu().numpy()
    confidences = results[0].boxes.conf.cpu().numpy()
    class_ids = results[0].boxes.cls.cpu().numpy().astype(int)

    detections = sv.Detections(
        xyxy=boxes,
        confidence=confidences,
        class_id=class_ids
    )

    greyhound_detections = filter_greyhound_detections(detections)
    annotated_frame = box_annotator.annotate(scene=frame, detections=greyhound_detections)
    video_writer.write(annotated_frame)

cap.release()
video_writer.release()
cv2.destroyAllWindows()

print(f"Processed video saved to {output_path}")


This script applies YOLOv8 object detection to a video, filters detections to focus on greyhounds, and saves the results to an annotated video file. The setup includes initializing the model, verifying directories, processing frames, and saving the output.